In [ ]:
# Project Overview  

This project explores the capabilities of different AI models, including **OpenAI’s GPT** and **Ollama**, to summarize online training content. It extracts material from Microsoft’s **Quantum Computing Fundamentals** learning path, processes it lesson by lesson, and generates concise summaries as well as overall course-level insights. Users can now interactively choose whether to summarize a single lesson, an entire module, or all lessons combined, and compare outputs from different models in real time.  

## Key Features  

- **Lesson-level Summaries**: Generates summaries for each individual lesson inside every module.  
- **Full-course Summaries**: Aggregates all lesson summaries into an overall training path summary.  
- **Dual Model Support**: Works with both **GPT** (via OpenAI) and **Ollama** backends.  
- **Interactive Interface**: Uses **Gradio** for a simple, user-friendly interface that allows users to:  
  - Select a model (GPT or Ollama)  
  - Choose lesson length/detail (short, medium, detailed)  
  - Pick language for the summaries  
  - Select a specific lesson or summarize all lessons at once  
- **Clean Markdown Output**: Summaries are formatted in Markdown, making results structured and easy to read.  

## Tech Stack  

- **Models**: GPT-4o-mini, Ollama  
- **Interface**: Gradio  
- **Language**: Python  
- **Libraries**: BeautifulSoup, OpenAI  

## Purpose  

This project demonstrates how AI can streamline understanding of technical documentation and online courses, not only by summarizing modules but also by breaking them down lesson by lesson. It highlights differences in performance and output quality between models while offering an interactive way for users to explore and compare results.  


In [ ]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI
import gradio as gr

# If you get an error running this cell, then please head over to the troubleshooting notebook!

In [ ]:
# Load environment variables  from .env file (not included)

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them")
else:
    print("API key found and looks good so far!")


In [ ]:
openai = OpenAI()

# If this doesn't work, try Kernel menu >> Restart Kernel and Clear Outputs Of All Cells, then run the cells from the top of this notebook down.


In [ ]:
# ollama api
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL_LLAMA = "llama3.2"

In [ ]:
!ollama pull llama3.2

In [ ]:
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [ ]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

        links = [a.get("href") for a in soup.find_all("a", href=True)]
        self.links = [link.strip() for link in links if link]

In [ ]:
# Create a system prompt function that can use different language and length 

def build_system_prompt(language="Spanish", length="short"):
    return f"""You are an assistant that analyzes the contents of a website and provides a {length} summary, ignoring text that might be navigation related.
    Respond in markdown, and respond in {language}.
    """
    
    
                        

In [ ]:
# Create a function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a {length} summary in {language} of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [ ]:

def messages_for(website, language, length):
    return [
        {"role": "system", "content": build_system_prompt(language, length)},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [ ]:
link_system_prompt = """
You are provided with a list of links found on a Microsoft Learn training page.

Decide which of the links are lessons within the Quantum Computing Fundamentals training path
(https://learn.microsoft.com/en-us/training/paths/quantum-computing-fundamentals/).

- Only include links that are actual lesson/module pages within this training path.
- Replace relative links (like /training/modules/...) with full https://learn.microsoft.com/... URLs.
- Ignore links to navigation, terms of service, privacy, blogs, or anything not part of this course.

You should respond in JSON in this format:

{
    "links": [
        {"type": "lesson", "url": "https://learn.microsoft.com/en-us/training/modules/intro-to-azure-quantum/1-introduction"},
        {"type": "lesson", "url": "https://learn.microsoft.com/en-us/training/modules/intro-to-azure-quantum/2-what-is-quantum-compute"}
    ]
}
"""


In [ ]:
def get_links_user_prompt(website):
    user_prompt = (
        f"Here are the raw links scraped from {website.url}.\n"
        "Only include links that are lesson/module pages for the Quantum Computing Fundamentals path.\n"
        "A valid module URL will contain '/training/modules/'.\n"
        "Return a JSON object with an array 'links' where each item has type:'lesson' and url: full https URL.\n"
        "Do not include Terms of Service, Privacy, navigation, blog, external marketing, error pages, 404 pages, or mailto links.\n\n"
        "Links (some may be relative):\n"
        + "\n".join(website.links)
    )
    return user_prompt


In [ ]:
import json

def get_links(url):
    website = Website(url)   # Website.links should be the raw href list
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
        ],
        response_format={"type": "json_object"}
    )
    # parse model output (response.choices[0].message.content is JSON text)
    result = response.choices[0].message.content
    return json.loads(result)


In [ ]:
import json

def get_links_ollama(url):
    website = Website(url)   # Website.links should be the raw href list
    response = ollama_via_openai.chat.completions.create(
        model= MODEL_LLAMA,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
        ],
        response_format={"type": "json_object"}
    )
    # parse model output (response.choices[0].message.content is JSON text)
    result = response.choices[0].message.content
    return json.loads(result)


In [ ]:
#call the OpenAI API. 

def summarize(url, language, length):
    website = Website(url)
    response = openai.chat.completions.create(
        model= "gpt-4o-mini",
        messages=messages_for(website, language, length)
    )
    return response.choices[0].message.content
    

In [ ]:
#call ollama 

def summarize_ollama(url, language, length):
    website = Website(url)
    response = ollama_via_openai.chat.completions.create(
        model= MODEL_LLAMA,
        messages=messages_for(website, language, length)
    )
    return response.choices[0].message.content
    

In [ ]:
#Summarize all the lessons in microsoft quantum computer training
from urllib.parse import urljoin

def summarize_training(path_url, language, length):
    # data = get_links(path_url) #returns json with links
    all_lessons = get_lesson_choices(path_url) #returns json with links
    # links = data.get("links", []) # extract list of lesson dicts
    print(f"Found {len(all_lessons)} lessons")

    all_summaries = []

    # for link in links:
    #     url = link["url"]
    #     title = link.get("title", url) #fallback if no title 

    for lesson_title, lesson_url in all_lessons[1:]:
        if lesson_url.startswith("//"):
            lesson_url = "https:" + lesson_url
        elif lesson_url.startswith("/"):
            lesson_url = urljoin("https://learn.microsoft.com", lesson_url)
        print(f"Summarizing {lesson_title} ({lesson_url})...")
        summary = summarize(lesson_url, language, length)
        all_summaries.append(f"### {lesson_title}\n{summary}\n")

    combined_prompt = "Here are summaries of each lesson:\n\n" + "\n".join(all_summaries)
    response = openai.chat.completions.create(
        model= "gpt-4o-mini",
        messages=[
            {"role": "system", "content": build_system_prompt(language, length)},
            {"role": "user", "content": "Please summarize the entire training path based on these lesson summaries:\n\n" + combined_prompt}
        ]
    )

    return "\n".join(all_summaries) + "\n\n## General Course Summary\n" + response.choices[0].message.content
    

In [ ]:
def get_lesson_choices(path_url):
    data = get_links(path_url)
    modules = data.get("links", [])
    all_lessons = []
    # filtered = []

    for module in modules:
        module_url = module.get("url")

        #fetch module page to get title
        module_site = Website(module_url)
        module_title = module_site.title or "Untitled Module"

        # go inside module page
        module_data = get_links(module_url)
        sublinks = module_data.get("links", [])


        for i, sub in enumerate(sublinks):
            lesson_title = sub.get("title", f"Lesson {i+1}")
            full_title = f"{module_title} - {lesson_title}"
            lesson_url = sub.get("url")
            all_lessons.append((full_title, lesson_url))

    return [("All Lessons", path_url)] + all_lessons

In [ ]:
#Summarize all the lessons in microsoft quantum computer training
from urllib.parse import urljoin

def summarize_training_ollama(path_url, language, length):
    all_lessons = get_lesson_choices(path_url)

    print(f"Found {len(all_lessons)-1} lessons")
    all_summaries = []

    for lesson_title, lesson_url in all_lessons[1:]:
         if lesson_url.startswith("//"):
             lesson_url = "https:" + lesson_url
         elif lesson_url.startswith("/"):
             lesson_url = urljoin("https://learn.microsoft.com", lesson_url)
             
         print(f"Summarizing {lesson_title} ({lesson_url})...")
         summary = summarize_ollama(lesson_url, language, length)
         all_summaries.append(f"### {lesson_title}\n{summary}\n")

    combined_prompt = "Here are summaries of each lesson:\n\n" + "\n".join(all_summaries)
    response = ollama_via_openai.chat.completions.create(
        model= MODEL_LLAMA,
        messages=[
            {"role": "system", "content": build_system_prompt(language, length)},
            {"role": "user", "content": "Please summarize the entire training path based on these lesson summaries:\n\n" + combined_prompt}
        ]
    )

    return "\n".join(all_summaries) + "\n\n## General Course Summary\n" + response.choices[0].message.content
    

In [ ]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url, language, length):
    summary = summarize(url, language, length)
    display(Markdown(summary))

In [ ]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary_all(url, language, length):
    summary = summarize_training(url, language, length)
    display(Markdown(summary))

In [ ]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary_ollama(url, language, length):
    summary = summarize_ollama(url, language, length)
    display(Markdown(summary))

In [ ]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary_ollama_all(url, language, length):
    summary = summarize_training_ollama(url, language, length)
    display(Markdown(summary))

In [ ]:
def get_lesson_dropdown():
    try:
        return ["All Lessons"] + get_lesson_choices("https://learn.microsoft.com/en-us/training/paths/quantum-computing-fundamentals/")
    except:
        return ["All Lessons"]

In [ ]:
def summarize_choose_model(model, language, length, lesson_choice):
    path_url = "https://learn.microsoft.com/en-us/training/paths/quantum-computing-fundamentals/"
    
    if lesson_choice == "All Lessons":
        # selected_index = None
        selected_url = path_url
        path_url = "https://learn.microsoft.com/en-us/training/paths/quantum-computing-fundamentals/"
        if model == "GPT":
            result = display_summary_all(path_url, language, length)
        elif model == "Ollama":
            result = display_summary_ollama_all(path_url, language, length)

    else:
        path_url = lesson_choice
            
    if model == "GPT":
        result = display_summary(path_url, language, length)
    elif model == "Ollama":
        result = display_summary_ollama(path_url, language, length)
    else:
        raise ValueError("Please choose a different option")
    return result    

In [ ]:
lesson_titles = get_lesson_dropdown()
view = gr.Interface(

    fn=summarize_choose_model,
    inputs=[gr.Dropdown(["GPT", "Ollama"], label="Select Model", value="GPT"), 
            gr.Dropdown(["English", "Spanish", "French", "Italian"], label="Select Language", value="English"), 
            gr.Dropdown(["Short","Medium", "Long"], label="Summary Length", value="Short"),
            gr.Dropdown(lesson_titles, label="Select Lesson", value="All Lessons")
            ],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()